In [ ]:
import gzip
import itertools
import os

import numpy as np
import pandas as pd
import plotly
import plotly.offline
import plotly.graph_objs as go
import requests
import sklearn
import sklearn.datasets
import sklearn.ensemble
import sklearn.metrics

try:
    import tqdm
except ModuleNotFoundError:
    pass

plotly.offline.init_notebook_mode(connected=True)

# implementing CADE in `python`

## introduction

### sources

I'm basing this off of conversations with ozan, mike, and wayne, as well as

+ [this paper](https://kdl.cs.umass.edu/papers/friedland-et-al-sdm2014.pdf)
+ [associated main page at umass](https://kdl.cs.umass.edu/display/public/Classifier-Adjusted+Density+Estimation+for+Anomaly+Detection+and+One-Class+Classification)
+ [kenny darrell's blog post](http://darrkj.github.io/blog/2014/may102014/)
+ [kenny darrell's R code](https://github.com/darrkj/CADE/blob/master/R/outlier.R)

from mike:

> You already know CADE, but let me give you my simple view of the process steps:
>
>   1. Limit the ABT to just the n records of interest with only the ID columns and the inputs, hiding the actual target
>   2. Add a column, tgt_anom_ind, the intermediate target variable, and assign all rows in the ABT a value of 0
>   3. Synthesize n additional rows (same number of rows as the real records) into the ABT, and assign tgt_anom_ind=1 to these artificial rows.  The ABT will have doubled in size.
>   4. Build a model that predicts tgt_anom_ind accurately.  Turns out, random forest has always worked extremely well for this, so please use that.  It typically gives an ROC AUC of 0.99 or so.
>   5. Score all the records in the ABT, adding a column of the probability that tgt_anom_ind=1, p_tgt_anom_ind which ranges from 0. to 1.0
>   6. Remove the synthetic records from the ABT, leaving just the original columns plus the new p_tgt_anom_ind
>   7. Graph the distribution of p_tgt_anom, and decide on a threshold value to call out anomalous records
> 
> Now, in this use case, we see how well we did against the CoverType4_ind true target.  Just report the average value of p_tgt_anom_ind for CoverType4_ind=1 vs. against CoverType4_ind=0.  I see a 5x difference.
> 
> The hard part is step 3 above, synthesizing n rows.  I will put that in a separate email.  I have my methods!
> 
> Following up on step 3, the "synthetic" observations. (Plus, I am attaching some example data, including the metadata.)
We get different answers (about which real observations were anomalous) depending on which method we choose--a point that will be made clear in class.
> 
> Here are three major methods for creating synthetic records:
> 
> + Uniform random distribution:
>     + For continuous real features, take the observed range, then assign synthetic values randomly (uniform random distribution) within that range.
>     + For nominal features, take the observed values and then assign synthetic values randomly equally among the nominal levels
> + Normal random distribution:
>     + For continuous real features, take the observed range mean and standard deviation, then assign synthetic values randomly (normal random distribution) for that distribution.  Then, you have an additional step:  cap and floor the synthetic value to be the min and max of the observed range.
>     + For nominal features, do the same as is described in the shuffled distribution below.
> + Shuffled distribution:
>     + Shuffle each feature vector randomly, then put them back together into a table.  Just like target shuffling, except that we do it for each of the features separately (with different random seeds).
>
> The hands on exercise will use methods 1 and 2, first with all the inputs and second with the known predictive features (elevation, horizontal distance to roadways, and 9am shade).  What we will see is that method 1 produces 5x lift over random, but method 2 produces no lift when all the inputs are used and actually produces negative lift when just the predictive inputs are used.
> 
> Lessons learned from hands-on exercise:
> 
> + CADE with synthetic records created with Uniform distribution typically produces a predictive anomaly score.  This is borne out in the papers on this and my own experience, and in this example gives a 5x lift.
> + CADE with other distributions may or may not be helpful.
> + Having labeled data is EXTREMELY helpful--dramatically (an order of magnitude or more) more helpful than the "back door" unsupervised methods (including cluster analysis, btw).
>
> Said another way, this demonstrates that an outlier score (e.g., the uniform CADE score) can be related to the target of interest in the expected way, but not reliably.  We should always remember that in fraud use cases, the fraudsters do their utmost to look "normal."  To the extent they are successful, unsupervised anomaly detection will not find them!  So, yes, we should always be on the lookout for anomalies, but this should be no substitute for labeled fraud pattern analysis and modeling.  Having said that, some anomaly scores (of which there are many different flavors) may well be among the many predictive features in a supervised fraud model, so unsupervised learning is a valuable tool in the fraud analyst's toolkit.
> 
> I am anxious to see how you code this up in Python!  I am not a Python programmer but learning...

## loading data

we're going to work with two datasets:

1. [the forest cover dataset](http://odds.cs.stonybrook.edu/forestcovercovertype-dataset/)
    + [uci landing page](https://archive.ics.uci.edu/ml/datasets/Covertype)
    + [direct download via dropbox](https://www.dropbox.com/s/awx8iuzbu8dkxf1/cover.mat?dl=0), not automated
2. a randomly generated 3d dataset
    + automated below

### forest cover

first, let's download the `gz` archive to our `/tmp` director

In [ ]:
tmpdir = os.path.join(os.sep, 'tmp')
if not os.path.isdir(tmpdir):
    os.makedirs(tmpdir)

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz'
fgzdownload = os.path.join(os.sep, 'tmp', os.path.basename(url))
resp = requests.get(url)
with open(fgzdownload, 'wb') as f:
    f.write(resp.content)

unzip that bad boy

In [ ]:
fcover = os.path.splitext(fgzdownload)[0]

with gzip.open(fgzdownload, 'rb') as fin, open(fcover, 'wb') as fout:
    fout.write(fin.read())

In [ ]:
dfcover = pd.read_csv(fcover, header=None)
dfcover.head()

add column names (going off info [here](https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info))

In [ ]:
attributes = [
    'Elevation',
    'Aspect',
    'Slope',
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points',
]
attributes = [_.lower() for _ in attributes]
wildernessareas = ['wilderness_area_{}'.format(i) for i in range(4)]
soiltypes = ['soil_type_{}'.format(i) for i in range(40)]
coverpredictors = attributes + wildernessareas + soiltypes
colnames = coverpredictors + ['covertype']

dfcover.columns = colnames
dfcover.head()

later on it will matter that some of these are continuous and others are categorical (the binary ones). let's encode that as the dtype (categorical as it is more generic than boolean):

In [ ]:
categoricals = wildernessareas + soiltypes + ['covertype']
for column in categoricals:
    dfcover.loc[:, column] = dfcover[column].astype('category')

dfcover.dtypes

### random 3d dataset

#### generating

In [ ]:
xrand, yrand = sklearn.datasets.make_blobs(
    n_samples=1000, n_features=3, centers=[[0, 0, 0]], random_state=1337
)
dfrand = pd.DataFrame(xrand, columns=['x', 'y', 'z'])
dfrand.loc[:, 'is_corner'] = 0
dfrand.head()

this dataset already has naturally occurring outliers, but let's also manually add 8 relatively extreme outliers

In [ ]:
dfcorners = pd.DataFrame(
    list(itertools.product([-2, 2], repeat=3)), 
    columns=['x', 'y', 'z']
)
dfcorners.loc[:, 'is_corner'] = 1
dfcorners.head()

In [ ]:
dfrand = pd.concat([dfrand, dfcorners]).reset_index(drop=True)
dfrand.head()

In [ ]:
dfrand.tail()

#### visualizing

In [ ]:
data = [
    go.Scatter3d(
        x=dfrand.x,
        y=dfrand.y,
        z=dfrand.z,
        mode='markers',
        marker={
            'opacity': 0.8, 
            'color': dfrand.is_corner,
            'colorscale': 'Portland'
        }
    )
]

layout = go.Layout(width=800, height=800)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

**one important note**: our goal in creating these corner elements *isn't* to distinguish the "fake" data from the "real" data, but rather to have some items we know ahead of time *should* be observerd, by CADE, to be anomalies.

even in the regular dataset there were already anomalous records -- you can see that visually -- but we are stacking the example with some simple cases in addition to the random ones.

## implementing CADE

### creating synthetic records

as mike outlined (see the above), there are three ways to generate synthetic records:

+ uniform
+ normal
+ shuffled

let's implement them

In [ ]:
NOMINAL_DTYPES = ['category']

def synthetic_generator_uniform(df):
    def synth(feature):
        if feature.dtype.name in NOMINAL_DTYPES:
            synth = sklearn.utils.resample(
                feature, n_samples=feature.shape[0]
            ).values
        else:
            synth = np.random.uniform(
                1.25 * feature.min(), 1.25 * feature.max(), feature.shape[0]
            )
        return synth
    
    return pd.DataFrame({
        colname: synth(feature) for (colname, feature) in df.iteritems()
    })[df.columns]


def synthetic_generator_normal(df):
    def synth(feature):
        if feature.dtype.name in NOMINAL_DTYPES:
            synth = sklearn.utils.shuffle(feature).values
        else:
            synth = np.random.normal(
                feature.mean(), feature.max(), feature.shape[0]
            )
        return synth
    
    return pd.DataFrame({
        colname: synth(feature) for (colname, feature) in df.iteritems()
    })[df.columns]


def synthetic_generator_shuffled(df):
    return pd.DataFrame({
        colname: sklearn.utils.shuffle(feature).values 
        for (colname, feature) in df.iteritems()
    })[df.columns]

In [ ]:
synth = synthetic_generator_uniform(dfcover)
synth.head()

## random distribution POC

let's take the random 3d dataset and walk through the algo one step at a time, following mike's 7-step outline

### abt with no target

Limit the ABT to just the n records of interest with only the ID columns and the inputs, hiding the actual target

In [ ]:
dfrand.head()

In [ ]:
dfrand.mean()

### add an anomaly metatarget 

Add a column, tgt_anom_ind, the intermediate target variable, and assign all rows in the ABT a value of 0

In [ ]:
dfrand.loc[:, 'is_real'] = 1
dfrand.head()

### create duplicate synthetic dataset

Synthesize n additional rows (same number of rows as the real records) into the ABT, and assign tgt_anom_ind=1 to these artificial rows.  The ABT will have doubled in size.

In [ ]:
randpredictors = ['x', 'y', 'z']
dfrandsynth = synthetic_generator_uniform(dfrand[randpredictors])
dfrandsynth.loc[:, 'is_real'] = 0
dfrandsynth.head()

In [ ]:
dfboth = pd.concat([dfrand, dfrandsynth]).reset_index(drop=True)
dfboth.head()

In [ ]:
dfboth.tail()

#### visualizing

In [ ]:
data = [
    go.Scatter3d(
        x=dftemp.x, y=dftemp.y, z=dftemp.z,
        mode='markers',
        name=name,
        marker={'opacity': opacity}
    )
    for (name, dftemp, opacity) in [
        ['real regular', dfboth[dfboth.is_corner == 0], 0.8],
        ['real corner', dfboth[dfboth.is_corner == 1], 0.8],
        ['syntehtic', dfboth[dfboth.is_real == 0], 0.4],
    ]
]

plotly.offline.iplot(data)

### predict metatarget

Build a model that predicts tgt_anom_ind accurately.  Turns out, random forest has always worked extremely well for this, so please use that.  It typically gives an ROC AUC of 0.99 or so.

In [ ]:
dfboth.head()

In [ ]:
x = dfboth[['x', 'y', 'z']]
y = dfboth.is_real

not sure if I should be training with a validation set or not. it feels, intuitively, that it defeats the point. we train it excessively well because the predictor score matters more than the generalizability

In [ ]:
rfc = sklearn.ensemble.RandomForestClassifier(n_estimators=1000, n_jobs=-1)

In [ ]:
rfc.fit(x, y)

### score abt

Score all the records in the ABT, adding a column of the probability that tgt_anom_ind=1, p_tgt_anom_ind which ranges from 0. to 1.0

In [ ]:
dfboth.loc[:, 'predicted_probability_is_real'] = rfc.predict_proba(x)[:, 1]
dfboth.head()

In [ ]:
sklearn.metrics.roc_auc_score(y_true=dfboth.is_real, y_score=dfboth.predicted_probability_is_real)

### look at only real records

Remove the synthetic records from the ABT, leaving just the original columns plus the new p_tgt_anom_ind
Graph the distribution of p_tgt_anom, and decide on a threshold value to call out anomalous records

In [ ]:
dfboth[dfboth.is_real == 1].sort_values(by='predicted_probability_is_real').head()

In [ ]:
dfboth[dfboth.is_corner == 1]

In [ ]:
dfdh = dfboth[dfboth.is_real == 1].sort_values(
    by='predicted_probability_is_real'
)

data = [
    go.Scatter(
        x=list(range(dfdh.shape[0])),
        y=dfdh.predicted_probability_is_real,
        mode='markers',
        marker={
            'opacity': 0.8, 
            'color': dfdh.is_corner,
            'size': np.where(dfdh.is_corner == 1, 15, 5),
            'colorscale': 'Portland',
        }
    )
]

plotly.offline.iplot(data)

in the plot above, the items on the lower left are the ones which were most confusing to our classifier -- that is, they were harder to distinguish from that uniformally distributed cloud of points than the bulk of points near the origin.

let's take, as our cutoff, the value of the least-confusing corner, and declare any record with a predicted probability value below that cutoff to be an anomaly

In [ ]:
cutoff = dfboth[dfboth.is_corner == 1].predicted_probability_is_real.max()

dfboth.loc[(dfboth.is_real == 1) & (dfboth.predicted_probability_is_real <= cutoff), 'is_anomaly'] = 1
dfboth.is_anomaly.fillna(0, inplace=True)
dfboth[dfboth.is_anomaly == 1].head()

how many anomalies is this?

In [ ]:
dfboth.is_anomaly.sum()

and now, to visualize them:

In [ ]:
dfplot = dfboth[dfboth.is_real == 1]
data = [
    go.Scatter3d(
        x=dfplot.x,
        y=dfplot.y,
        z=dfplot.z,
        mode='markers',
        marker={
            'opacity': 0.8, 
            'color': dfplot.is_anomaly,
            'colorscale': 'Portland'
        }
    )
]

layout = go.Layout(width=800, height=800)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

not too bad!

### a general framework

let's pick an interface and implement the above as a function

In [ ]:
def cade(dfreal, predictorcols, synthfunc=synthetic_generator_uniform,
         model=None, plotprobs=True, inplace=True):
    """perform the CADE outlier detection algorithm
    
    args:
        dfreal (pandas DataFrame): a dataframe of real records to which we wish 
            to apply the CADE algorithm
        predictorcols (list): a list of string column names of the predictor 
            columns in `dfreal`
        synthfunc (func): a function which can take a pandas dataframe and 
            generate synthetic data. the output will be concatenated with the
            values in `dfreal` and used as the training set for `model`
        model (None or scikit learn model): an object which has a `fit` and
            `predict_proba` method (*a la* a scikit learn model). this model is
            used by the CADE algorithm when it predicts whether a record is real 
            or synthetic. if no model is passed, we will use a 1000-tree random
            forest by default
        plotprobs (bool): whether or not to plot the probabilities a given 
            record is real or synthetic (as predicted by `model`)
        inplace (bool): whether or not to update the passed `dfreal` inplace as 
            we create real and synthetic records (will change the shape of 
            `dfreal`)
    
    returns:
        dfboth (pandas DataFrame): a dataframe which contains the original 
            `dfreal` records as well as synthetically generated records 
        rocauc (float): the ROC AUC of the CADE prediction
    
    raises:
        ValueError
        
    """
    # build synth dataset
    print('building synthetic records')
    df = dfreal if inplace else dfreal.copy()
    df.loc[:, 'is_real'] = 1

    dfsynth = synthetic_generator_uniform(df[predictorcols])
    dfsynth.loc[:, 'is_real'] = 0

    dfboth = pd.concat([df, dfsynth]).reset_index(drop=True)

    # get a target and predictor array and fit the user-provided model
    print('training a model on real and synthetic data')
    x = dfboth[predictorcols]
    y = dfboth.is_real

    if model is None:
        model =  sklearn.ensemble.RandomForestClassifier(
            n_estimators=1000, n_jobs=-1
        )
    model.fit(x, y)

    # add predicted probability of being real to our real + synthetic dataset
    dfboth.loc[:, 'predicted_prob_is_real'] = model.predict_proba(x)[:, 1]

    rocauc = sklearn.metrics.roc_auc_score(
        y_true=dfboth.is_real,
        y_score=dfboth.predicted_prob_is_real
    )
    
    # visualize the predicted probabilities if the user requests it
    if plotprobs:
        print('generating predicted probability plot')
        
        if dfboth.shape[0] > 10000:
            # switch to a box and whiskers version
            data = [
                go.Box(
                    y=chunk.predicted_prob_is_real,
                    name='is_real = {}'.format(isrealval),
                    boxpoints='outliers'
                )
                for (isrealval, chunk) in dfboth.groupby('is_real')
            ]

            layout = go.Layout(
                title="predicted probabilities for real and synthetic records",
                showlegend=True
            )
        else:
            # good ol' devil's horn plot
            dfboth = dfboth.sort_values(by='predicted_prob_is_real')

            data = [
                go.Scatter(
                    x=list(range(dfboth.shape[0])),
                    y=dfboth.predicted_prob_is_real,
                    mode='markers',
                    marker={
                        'opacity': 0.8, 
                        'color': dfboth.is_real,
                        'colorscale': 'Portland'
                    }
                )
            ]

            layout = go.Layout(
                title="predicted probability for real and synthetic records",
                xaxis={'title': 'predicted probability sort order index'},
                yaxis={'title': 'predicted probability'},
                showlegend=True
            )
        
        fig = go.Figure(data=data, layout=layout)
        plotly.offline.iplot(fig)

    return dfboth, rocauc


def declare_anomaly(dfreal, cutoff=None, numanom=None, anomalycol='is_anomaly',
                    cadepredictioncol='predicted_prob_is_real'):
    """add a column to `dfreal` indicating whether or not a record is considered 
    to be an anomaly. 
    
    given a dataframe of *only real* records (i.e. synthetics already dropped),
    we add a column `anomalycol` that is a 1 if a record is an anomaly based on
    a CADE predicted probability column `cadepredictioncol`.
    
    one (and only one) of `cutoff` and `numanom` can be provided
    
    args:
        dfreal (pandas DataFrame): dataframe of *only real records*. you *must* 
            have dropped synthetic records from `dfreal` in order for this 
            function to make sense
        cutoff (None or float): the numerical value of the cutoff predicted 
            probability (every record with a predicted probability less than 
            this value is declared an anomaly)
        numanom (None or int): integer number of records you wish to declare 
            annomalous
        anomalycol (str): column name of output anomaly indicator column
        cadepredictioncol (str): existing column name of predicted probability 
            values
            
    returns:
        dfreal (pandas DataFrame): the dataframe with an additional column 
            indicating annomalous records
    
    raises:
        ValueError
    """
    if (cutoff is None) == (numanom is None):
        raise ValueError(
            "you must provide one and only one of `cutoff` or `numanom`"
        )
    
    dfreal = dfreal.sort_values(by=cadepredictioncol, ascending=True)
    cutoff = cutoff or dfreal.iloc[int(numanom + 1), :][cadepredictioncol]
    
    dfreal.loc[dfreal[cadepredictioncol] < cutoff, anomalycol] = 1
    dfreal.is_anomaly.fillna(0, inplace=True)
    
    return dfreal


def visualize_binary_groups(dfreal, groupcol='is_anomaly', plotcols=['x', 'y', 'z']):
    """visualize records as anomalies or not anomalies
    
    args:
        dfreal (pandas DataFrame): dataframe of *only real records*. you *must* 
            have dropped synthetic records from `dfreal` in order for this 
            function to make sense
        groupcol (str): column name of distinguishing indicator column (will be 
            used to set color of scatter points)
        plotcols (list): list of either two or three columns in the provided 
            dataset `dfreal` we should visualize. this will create a 2d or 3d
            plot depending on what is passed
        
    returns:
        None
        
    raises:
        None
        
    """
    coords = {k: dfreal[col] for (k, col) in zip('xyz', plotcols)}
    if len(plotcols) == 2:
        plotobj = go.Scatter
    elif len(plotcols) == 3:
        plotobj = go.Scatter3d
    else:
        raise ValueError('we can only visualize 2 or 3 columns at a time')
    
    data = [
        plotobj(
            mode='markers',
            marker={
                'opacity': 0.8, 
                'color': dfreal[groupcol],
                'colorscale': 'Portland'
            },
            **coords
        )
    ]

    layout = go.Layout(
        height=800,
        width=800,
        title="records colored by feature '{}'".format(groupcol)
    )

    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)
    
    
def example_rand_with_corners(n_samples=1000, n_features=3, random_state=1337,
                             colnames=['x', 'y', 'z']):
    """build our example randomly generated dataset with anomalous corner points
    
    args:
        n_samples (int): number of random records to generate
        n_features (int): number of columns
        random_state (int): RNG seed number
        colnames (list): list of column names for output dataset (must have 
            `n_features` elements)
            
    returns:
        dfrand (pandas DataFrame): dataframe of random normally distributed 
            records in `n_features` dimensions
            
    raises:
        None
        
    """
    # make regular random values
    xrand, yrand = sklearn.datasets.make_blobs(
        n_samples=n_samples,
        n_features=n_features,
        centers=[[0 for i in range(n_features)]],
        random_state=random_state
    )
    dfrand = pd.DataFrame(xrand, columns=colnames)
    dfrand.loc[:, 'is_corner'] = 0

    # make "corner" values
    dfcorners = pd.DataFrame(
        list(itertools.product([-2, 2], repeat=n_features)), 
        columns=colnames
    )
    dfcorners.loc[:, 'is_corner'] = 1
    
    dfrand = pd.concat([dfrand, dfcorners]).reset_index(drop=True)
    return dfrand

so we can reproduce all of the above with the following function calls:    

In [ ]:
dfrand = example_rand_with_corners()

dfboth, rocauc = cade(
    dfreal=dfrand,
    predictorcols=['x', 'y', 'z']
)

print('our CADE model had a ROC AUC of {}'.format(rocauc))

In [ ]:
dfreal = dfboth[dfboth.is_real == 1].copy()
dfreal = declare_anomaly(
    dfreal=dfreal,
    cutoff=None,
    numanom=100,
)
dfboth.head()

In [ ]:
visualize_binary_groups(
    dfreal,
    groupcol='is_anomaly', 
    plotcols=['x', 'y', 'z']
)

### applying our framework to the forest cover data set

In [ ]:
dfrand = example_rand_with_corners()

# will use a smaller random forest for time constraints
rfc = sklearn.ensemble.RandomForestClassifier(n_estimators=3, n_jobs=-1)
dfboth, rocauc = cade(
    dfreal=dfcover,
    predictorcols=coverpredictors,
    model=rfc
)

print('our CADE model had a ROC AUC of {}'.format(rocauc))

In [ ]:
dfreal = dfboth[dfboth.is_real == 1].copy()
dfreal = declare_anomaly(
    dfreal=dfreal,
    cutoff=None,
    numanom=100,
)
dfreal.head()

In [ ]:
dfreal[dfreal.is_anomaly == 1]